### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
성능 개선을 위한 테스트에서는 전체를 대상으로 인덱싱하지 않고, 5개 인덱싱 코드 실행 권장(주석 해제)

In [11]:
from preprocess.pp_basic import docs, BASE_DIR, EVAL_DIR, GOLD_EVIDENCE_CSV, GOLD_FIELDS_JSONL, RAW_DIR
import preprocess.pp_v4 as pp

In [12]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

In [13]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

In [14]:
# 임베딩 및 인덱스 만들기
# 한국어 임베딩 모델
embed_model_name = "nlpai-lab/KoE5"
embed_model = SentenceTransformer(embed_model_name)

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 2439.83it/s, Materializing param=pooler.dense.weight]                               


In [15]:
# 질문 리스트 (key, 질문)
queries = [
    ("project_name", "사업(용역)명은 무엇인가?"),
    ("agency", "발주 기관(수요기관)은 어디인가?"),
    ("purpose", "사업 목적(추진 배경)은 무엇인가?"),
    ("budget", "총 사업 예산(사업비)은 얼마인가?"),
    ("contract_type", "계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?"),
    ("deadline", "입찰/제안서 제출 마감일시는 언제인가?"),
    ("duration", "사업 수행 기간은 얼마나 되는가?"),
    ("requirements_must", "필수 요구사항(기능/성능/보안 등)은 무엇인가?"),
    ("eval_items", "평가 항목(기술/가격 등) 구성은 어떻게 되는가?"),
    ("price_eval", "가격 평가 방식(최저가/협상 등)은 무엇인가?"),
    ("eligibility", "참가 자격 요건(면허/실적/인증/등급)은 무엇인가?"),
]

def build_query_prompt(queries):
    return "\n".join(
        f"{i}. {q}"
        for i, (_, q) in enumerate(queries, start=1)
    )

In [16]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

[분석 규칙]
- 추측 금지, 문서에 명시된 내용만 사용
- 문서에 없으면 반드시 "NOT_FOUND"
- **출력은 질문 개수와 동일한 줄 수**
- **각 줄에는 답변 텍스트만 작성**
- 번호, 질문 문장, '답:', 기호, 설명을 절대 포함하지 말 것

[질문 목록]
{questions}

[컨텍스트]
{context}

질문 순서대로 답변만 한 줄씩 출력하라.
"""

### 배치 질문 응답

질문 리스트를 한 번에 임베딩/검색하고, top-k 청크를 합쳐 LLM에 넣어 답변을 생성합니다.
`return_indices=True`일 때는 검색된 청크 인덱스도 반환합니다.

In [17]:
def answer_batch(index, model, chunks, queries, top_k: int = 15, return_indices: bool = False) -> list[str]:
    q_list = [q for _, q in queries]
    q_emb = model.encode(q_list)
    q_mean = q_emb.mean(axis=0, keepdims=True)
    _, I = index.search(q_mean.astype("float32"), top_k)
    context = "".join(chunks[i] for i in I[0])

    prompt = RFP_PROMPT.format(questions=build_query_prompt(queries), context=context)
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )
    text = (resp.choices[0].message.content or "").strip()
    answers = [line.strip() for line in text.splitlines() if line.strip()]
    if return_indices:
        return answers, [int(i) for i in I[0]]
    return answers

In [18]:
# # 쿼리 답변
# def answer(index, embed_model, chunks, queries, top_k: int = 15) -> dict:
#     # 1. 검색용 대표 질문
#     search_query = "RFP 주요 사업 정보 요약"
#     q_emb = embed_model.encode([search_query], convert_to_numpy=True)
#     _, I = index.search(q_emb.astype("float32"), top_k)
#     context = "\n\n".join(chunks[i] for i in I[0])
#     questions_text = build_query_prompt(queries)
#     prompt = RFP_PROMPT.format(
#         context=context,
#         questions=questions_text
#     )
#     # 2. OpenAI API 호출
#     resp = client.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
#             {"role": "user", "content": prompt},
#         ],
#         max_tokens=2000,
#         temperature=0.0,
#     )
#     raw_answer = resp.choices[0].message.content.strip()
#     # 3. key: value 매핑
#     results = {}
#     for (key, _), line in zip(queries, raw_answer.splitlines()):
#         line = line.strip()
#         if line:
#             results[key] = line
#         else:
#             results[key] = "명시 없음"
#     return results

In [19]:
# 데이터 폴더 설정
print(f"발견된 PDF: {len(docs)}개")
for i, doc in enumerate(docs):
    print(f"{i}: {doc.name}")

발견된 PDF: 100개
0: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
1: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
2: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
3: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
4: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
6: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
7: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
8: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
9: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
10: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
11: 경희대학교_[입찰공고] 산학협력단 정보시스

## RAG 평가 (docs_api_summary 베이스라인 그대로 + gold로 점수 측정)

In [24]:
import json
import pandas as pd
from tqdm.auto import tqdm
from rapidfuzz import fuzz

In [25]:
# 평가 데이터 경로
print("BASE_DIR:", BASE_DIR)
print("RAW_DIR:", RAW_DIR, "exists:", RAW_DIR.exists())
print("EVAL_DIR:", EVAL_DIR, "exists:", EVAL_DIR.exists())

BASE_DIR: /Users/won/dev/00_codeit/0_mission/200_DL_RAG
RAW_DIR: /Users/won/dev/00_codeit/0_mission/200_DL_RAG/data/raw/files exists: True
EVAL_DIR: /Users/won/dev/00_codeit/0_mission/200_DL_RAG/data/raw/eval exists: True


### 평가용 정답 데이터 불러오기

In [26]:
def load_gold_fields_jsonl(path: Path) -> pd.DataFrame:
    rows = []
    with path.open("r", encoding="utf-8", errors="replace") as f:
        for line in f:
            if not line.strip():
                continue
            rows.append(json.loads(line))
    out = []
    for r in rows:
        iid = r["instance_id"]
        doc_id = r.get("doc_id", "")
        fields = r.get("fields", {}) or {}
        for k, v in fields.items():
            out.append({"instance_id": iid, "doc_id": doc_id, "field": k, "gold": v})
    return pd.DataFrame(out)


gold_evidence_df = pd.read_csv(GOLD_EVIDENCE_CSV)
gold_fields_df = load_gold_fields_jsonl(GOLD_FIELDS_JSONL)

display(gold_evidence_df.head(3))
display(gold_fields_df.head(3))

,instance_id,doc_id,page_start,page_end,anchor_text
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,4.0,4.0,사업개요
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,1.0,1.0,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,3.0,3.0,추진배경 및 방향


,instance_id,doc_id,field,gold
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,project_name,벤처확인종합관리시스템 기능 고도화
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,agency,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,purpose,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능..."


### Retrieval 평가(Anchor 기반)

`gold_evidence.csv`의 `anchor_text`가 top-k 청크 안에 포함되는지로 retrieval을 평가합니다.
- `recall@k`: top-k 안에서 anchor가 한 번이라도 나오면 1
- `mrr@k`: 첫 hit의 역순위(없으면 0)

In [27]:
def build_gold_anchor_map(df: pd.DataFrame) -> dict[str, list[str]]:
    m: dict[str, list[str]] = {}
    for _, r in df.iterrows():
        iid = str(r["instance_id"])
        anchor = str(r.get("anchor_text", "") or "").strip()
        if anchor:
            m.setdefault(iid, []).append(anchor)
    return m

GOLD_ANCHOR = build_gold_anchor_map(gold_evidence_df)


def eval_retrieval_by_anchor(chunks: list[str], idxs: list[int], anchors: list[str]) -> dict[str, float]:
    hit_rank = None
    for rank, ci in enumerate(idxs, start=1):
        if 0 <= int(ci) < len(chunks):
            c = chunks[int(ci)]
            if any(a in c for a in anchors):
                hit_rank = rank
                break
    return {"recall": 1.0 if hit_rank else 0.0, "mrr": (1.0 / hit_rank) if hit_rank else 0.0}

In [28]:
# 베이스라인 질문 리스트 (docs_api_summary의 queries 사용)
# queries 변수는 기존 노트북 셀에 정의되어 있어야 합니다.
BASELINE_QUERIES = queries
BASELINE_QUESTIONS = dict(BASELINE_QUERIES)

### 정답 유사도 평가 함수

In [29]:
def eval_gen(pred: str, gold: str | None, threshold: int = 80) -> dict[str, float]:
    pred = (pred or "").strip()
    fill = 1.0 if pred and pred.lower() not in {"", "없음"} else 0.0
    if gold is None or str(gold).strip() == "":
        return {"fill": fill, "match": np.nan, "sim": np.nan}
    gold = str(gold).strip()
    sim = fuzz.token_set_ratio(pred, gold)
    return {"fill": fill, "match": 1.0 if sim >= threshold else 0.0, "sim": float(sim)}

In [30]:
SIM_THRESHOLD = 80
TOP_K = 15

In [33]:
# docs_api_summary에서 만든 docs 리스트 사용
# docs = get_pdf_paths(RAW_DIR) 가 이미 실행되어 있어야 합니다.
DOC_PATHS = docs
for p in DOC_PATHS:
    print(" -", p.name)

 - (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
 - (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
 - (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
 - (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
 - 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
 - BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
 - KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
 - 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
 - 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
 - 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
 - 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
 - 경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용여

### 문서 단위 평가

- generation: `gen_fill_rate`, `gen_match_rate`, `gen_avg_similarity`
- retrieval: `retrieval_recall@k`, `retrieval_mrr@k` (anchor 기반)
- score: 위 지표의 가중합

In [46]:
import unicodedata
def run_eval_for_doc(doc_path: Path) -> pd.DataFrame:
    doc_name = doc_path.name
    doc_name = unicodedata.normalize("NFC", doc_path.name)

    qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()
    if len(qdf) == 0:
        print("⚠️ no gold fields:", doc_name)
        return pd.DataFrame()

    # 베이스라인 청킹/인덱싱
    model = embed_model

    index, chunks = pp.gen_input(doc_path, embed_model)


    # 배치 답변 생성 (베이스라인 질문 순서)
    answers, idxs = answer_batch(index, model, chunks, BASELINE_QUERIES, top_k=TOP_K, return_indices=True)
    answer_map = {k: answers[i] if i < len(answers) else "" for i, (k, _) in enumerate(BASELINE_QUERIES)}


    # 평가
    r_list, g_list = [], []
    for _, row in qdf.iterrows():
        iid = str(row["instance_id"])
        field = str(row["field"])
        gold = row["gold"]
        pred = answer_map.get(field, "")

        anchors = GOLD_ANCHOR.get(iid, [])
        if anchors:
            r_list.append(eval_retrieval_by_anchor(chunks, idxs, anchors))
        else:
            r_list.append({"recall": np.nan, "mrr": np.nan})

        g_list.append(eval_gen(pred, gold, threshold=SIM_THRESHOLD))

    rows = [{
        "doc_id": doc_name,
        "n_questions_total": len(qdf),
        "n_questions_with_evidence": int(sum(1 for x in qdf["instance_id"] if str(x) in GOLD_ANCHOR)),
        "retrieval_recall@k": float(np.nanmean([x["recall"] for x in r_list])),
        "retrieval_mrr@k": float(np.nanmean([x["mrr"] for x in r_list])),
        "n_questions_with_gold_fields": len(qdf),
        "gen_fill_rate": float(np.nanmean([x["fill"] for x in g_list])),
        "gen_match_rate": float(np.nanmean([x["match"] for x in g_list])),
        "gen_avg_similarity": float(np.nanmean([x["sim"] for x in g_list])),
    }]
    return pd.DataFrame(rows)

In [51]:
all_results = []
for doc_path in DOC_PATHS[:5]:
    all_results.append(run_eval_for_doc(doc_path))

results_df = pd.concat([d for d in all_results if len(d)], ignore_index=True) if all_results else pd.DataFrame()

if len(results_df):
    results_df["score"] = (
        0.5 * results_df["retrieval_mrr@k"].fillna(0) +
        0.2 * results_df["retrieval_recall@k"].fillna(0) +
        0.3 * results_df["gen_match_rate"].fillna(0)
    )

display(results_df)

if len(results_df):
    out_path = BASE_DIR / "outputs" / "eval_integrated.csv"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    results_df.to_csv(out_path, index=False, encoding="utf-8-sig")
    display(results_df.sort_values(["retrieval_mrr@k", "gen_match_rate"], ascending=[False, False]).head(20))
    print("Saved:", out_path)

⚠️ no gold fields: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf


,doc_id,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,21,21,0.285714,0.114286,21,0.52381,0.095238,21.113656,0.142857
1,(사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...,21,21,0.761905,0.184524,21,0.52381,0.238095,31.925255,0.316071
2,(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf,21,21,0.857143,0.432540,21,0.52381,0.238095,35.168419,0.459127
3,2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf,21,21,0.333333,0.140136,21,0.52381,0.095238,16.004409,0.165306


,doc_id,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
2,(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf,21,21,0.857143,0.432540,21,0.52381,0.238095,35.168419,0.459127
1,(사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...,21,21,0.761905,0.184524,21,0.52381,0.238095,31.925255,0.316071
3,2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf,21,21,0.333333,0.140136,21,0.52381,0.095238,16.004409,0.165306
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,21,21,0.285714,0.114286,21,0.52381,0.095238,21.113656,0.142857


Saved: /Users/won/dev/00_codeit/0_mission/200_DL_RAG/outputs/eval_integrated.csv


### 질문 샘플 및 답변

In [56]:
# 질문 샘플 + 모델 답변 보기 (문서 1개 기준)
doc_name = results_df.sort_values("gen_match_rate").iloc[0]["doc_id"]  # 예시: 성능 낮은 문서
sample_n = 5

# 문서 선택
doc_path = next(p for p in DOC_PATHS 
                if unicodedata.normalize("NFC", p.name) == unicodedata.normalize("NFC", doc_name))


# 준비
qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()

index, chunks = pp.gen_input(doc_path, embed_model)
model = embed_model

# 배치 답변
answers, idxs = answer_batch(index, model, chunks, BASELINE_QUERIES, top_k=TOP_K, return_indices=True)
answer_map = {k: answers[i] if i < len(answers) else "" for i, (k, _) in enumerate(BASELINE_QUERIES)}

# 샘플 출력
rows = []
for _, row in qdf.head(sample_n).iterrows():
    field = str(row["field"])
    gold = row["gold"]
    pred = answer_map.get(field, "")
    question = dict(BASELINE_QUERIES).get(field, "")
    rows.append({"field": field, "question": question, "gold": gold, "pred": pred})

display(pd.DataFrame(rows))

,field,question,gold,pred
0,project_name,사업(용역)명은 무엇인가?,벤처확인종합관리시스템 기능 고도화,2024년 「벤처확인종합관리시스템 기능 고도화」 용역사업
1,agency,발주 기관(수요기관)은 어디인가?,벤처기업확인기관,NOT_FOUND
2,purpose,사업 목적(추진 배경)은 무엇인가?,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능...",본 사업은 개발서버를 제공하지 않음.
3,budget,총 사업 예산(사업비)은 얼마인가?,"352,000,000원","352,000,000원(부가가치세 포함)"
4,contract_type,계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?,제한경쟁입찰,제한경쟁입찰(협상에 의한 계약)


In [59]:
# 질문 샘플 + 모델 답변 보기 (문서 1개 기준)
doc_name = "(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf"
sample_n = 5

# 문서 선택
doc_path = next(p for p in DOC_PATHS 
                if unicodedata.normalize("NFC", p.name) == unicodedata.normalize("NFC", doc_name))

# 준비
qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()
text = pp.extract_text(doc_path)
chunks = pp.chunk(text)
index, chunks = pp.build_index(chunks, embed_model)
model = embed_model

# 배치 답변
answers, idxs = answer_batch(index, model, chunks, BASELINE_QUERIES, top_k=TOP_K, return_indices=True)
answer_map = {k: answers[i] if i < len(answers) else "" for i, (k, _) in enumerate(BASELINE_QUERIES)}

# 샘플 출력
rows = []
for _, row in qdf.head(sample_n).iterrows():
    field = str(row["field"])
    gold = row["gold"]
    pred = answer_map.get(field, "")
    question = dict(BASELINE_QUERIES).get(field, "")
    rows.append({"field": field, "question": question, "gold": gold, "pred": pred})

display(pd.DataFrame(rows))

,field,question,gold,pred
0,project_name,사업(용역)명은 무엇인가?,벤처확인종합관리시스템 기능 고도화,복수의결권 및 스톡옵션 시스템 구축 고도화 용역사업
1,agency,발주 기관(수요기관)은 어디인가?,벤처기업확인기관,정보화 사업을 담당하는 행정기관 및 공공기관
2,purpose,사업 목적(추진 배경)은 무엇인가?,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능...","벤처기업법에 따른 복수의결권주식, 스톡옵션 및 성과조건부주식의 기능 고도화 및 이관"
3,budget,총 사업 예산(사업비)은 얼마인가?,"352,000,000원",NOT_FOUND
4,contract_type,계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?,제한경쟁입찰,협상에 의한 계약
